In [ ]:
import pandas as pd


In [ ]:
#I am giving up on the API due to data limits, irregularities, etc
#Instead I am downloading files for avg temp, min temp, max temp and precipitation

#Data source is: https://www.ncdc.noaa.gov/cag/county/mapping/

#If we want snowfall, we can use the following link, but it will require us to download one
#file per month per year so this will be 60 files 
#https://www.ncdc.noaa.gov/snow-and-ice/daily-snow/

In [ ]:
#Load in all files
avg_temps = pd.read_csv('/work/assets/ca_avg_temp.csv',skiprows=3,usecols=[0,1,2,3])
min_temps = pd.read_csv('/work/assets/ca_min_temp.csv',skiprows=3,usecols=[0,1,2,3])
max_temps = pd.read_csv('/work/assets/ca_max_temp.csv',skiprows=3,usecols=[0,1,2,3])
precip_amts = pd.read_csv('/work/assets/ca_precip.csv',skiprows=3,usecols=[0,1,2,3])

In [ ]:
precip_amts.head()

,Location ID,Location,Date,Value
0,CA-001,Alameda County,189501,8.43
1,CA-001,Alameda County,189502,2.09
2,CA-001,Alameda County,189503,1.97
3,CA-001,Alameda County,189504,1.75
4,CA-001,Alameda County,189505,1.20


In [ ]:
#Check for missing data
print(len(avg_temps[avg_temps['Value']==-99]))
print(len(min_temps[min_temps['Value']==-99]))
print(len(max_temps[max_temps['Value']==-99]))
print(len(precip_amts[precip_amts['Value']==-99]))

0
0
0
0


In [ ]:
#Update column names
avg_temps = avg_temps.rename(columns={'Value':'avg_temp'})
min_temps = min_temps.rename(columns={'Value':'min_temp'})
max_temps = max_temps.rename(columns={'Value':'max_temp'})
precip_amts = precip_amts.rename(columns={'Value':'total_precip_amt'})

#Remove the years we don't care about
avg_temps['year']=avg_temps['Date'].astype(str).str[0:4]
min_temps['year']=min_temps['Date'].astype(str).str[0:4]
max_temps['year']=max_temps['Date'].astype(str).str[0:4]
precip_amts['year']=precip_amts['Date'].astype(str).str[0:4]

avg_temps = avg_temps[avg_temps['year'].between('2014','2019')]
min_temps = min_temps[min_temps['year'].between('2014','2019')]
max_temps = max_temps[max_temps['year'].between('2014','2019')]
precip_amts = precip_amts[precip_amts['year'].between('2014','2019')]

#Remove the date field
avg_temps = avg_temps.drop(columns=['Date'])
min_temps = min_temps.drop(columns=['Date'])
max_temps = max_temps.drop(columns=['Date'])
precip_amts = precip_amts.drop(columns=['Date'])

#Group by year
avg_grouped = avg_temps.groupby(by=['Location ID','Location','year'],as_index=False).mean()
min_grouped = min_temps.groupby(by=['Location ID','Location','year'],as_index=False).min()
max_grouped = max_temps.groupby(by=['Location ID','Location','year'],as_index=False).max()
precip_grouped = precip_amts.groupby(by=['Location ID','Location','year'],as_index=False).sum()

#Bring all values into one dataframe
all_vals = avg_grouped.merge(min_grouped,on=['Location ID','year','Location']) \
        .merge(max_grouped,on=['Location ID','year','Location']) \
        .merge(precip_grouped,on=['Location ID','year','Location'])

In [ ]:
all_vals.head()

,Location ID,Location,year,avg_temp,min_temp,max_temp,total_precip_amt
0,CA-001,Alameda County,2014,61.950000,42.0,85.0,19.88
1,CA-001,Alameda County,2015,61.050000,39.2,84.0,10.39
2,CA-001,Alameda County,2016,60.366667,39.1,83.8,22.55
3,CA-001,Alameda County,2017,60.500000,39.4,86.8,24.97
4,CA-001,Alameda County,2018,59.808333,40.5,85.9,16.57


In [ ]:
#Should be 58 counties * number of years returned, so right now, 6, = 348
len(all_vals)

348

In [ ]:
#Write data out to be used with other county variables
all_vals.to_csv('/work/cleaned-csvs/ca_weather.csv',index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f6c76417-5fde-42f3-8920-755838dec3fa' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>